# Two-wave RPS Algorithm

In [61]:
import numpy as np
import matplotlib.pyplot as plt
from rashomon.hasse import enumerate_policies
from rashomon.aggregate import RAggregate
from first_wave import compute_boundary_probs, allocate_wave, assign_first_wave_treatments
from data_gen import get_beta_underlying_causal, generate_outcomes

## 1. First-wave allocation

In [64]:
# get lattice
M = 4
R = 3
R_vec = np.full(M, R) if np.isscalar(R) else np.array(R) # allow for heterogeneity in levels
assert R_vec.shape == (M,)
policies = enumerate_policies(M, R)
K = len(policies)
print(f"Found K = {K} policies (each policy is an {M}-tuple).")
H = 5  # sparsity parameter used inside compute_boundary_probs TODO choice
n1 = 500  # total first‐wave sample size

Found K = 81 policies (each policy is an 4-tuple).


**Compute first‐wave allocation**: We need R_i for each feature (here R_i = R for i=0,…,M-1), then we call `compute_boundary_probs` -> `allocate_first_wave`. We get `n1_alloc`: an array of length K summing to n1.

In [65]:
boundary_probs = compute_boundary_probs(policies, R, H)
n1_alloc = allocate_wave(boundary_probs, n1)
print(f"First‐wave allocation sums to {int(n1_alloc.sum())} (should be {n1}).")

First‐wave allocation sums to 500 (should be 500).


## 2. Simulating first-wave outcomes

We generate a np.array `beta` of true effects for each node. We pass our lattice `policies`, `M` and `R`, and then specify a `kind` of underlying causal model.

There are a range of options, all of which are continuous and non-trivial: they exhibit locally correlated effects and avoid brittle cancellations in effects. The options range from simple (polynomial, gaussian, basic interaction) to complex (radial basis function, mimic of a simple neural-net-like function)

In [66]:
beta = get_beta_underlying_causal(policies, M, R, kind="gauss_sin")

In [67]:
# Not in use: different distribution for each true pool from a random 'true' partition sigma_true. Not used in this simulation due to our specifications on the underlying causal model (e.g. continuous, locally correlated effects, etc). Also needs changes on how it constructs a true partition.

# partition_seed = 123
# sigma_true, pi_pools_true, pi_policies_true = generate_true_partition(policies, R,random_seed=partition_seed)
# beta = get_beta_piecewise(policies, sigma_true, pi_pools_true, pi_policies_true, 0.5, 1, 10)

**Get outcomes**: we now track the first-wave assignment and generate the outcomes with additional noise

In [68]:
# now build first-wave assignment vector D
D1 = assign_first_wave_treatments(n_alloc=n1_alloc) # TODO check dimensions here
N1 = D1.shape[0]

print("Length of D1:", N1)  # should equal sum n1_alloc == n1

Length of D1: 500


In [69]:
# generate outcomes y1
sigma_noise = 5
outcome_seed = 53
y1 = generate_outcomes(D=D1, beta=beta, sigma_noise=sigma_noise, random_seed=outcome_seed)

print("Overall mean outcome:", np.mean(y1))
print("Overall std outcome:", np.std(y1))

Overall mean outcome: -0.06186879481060036
Overall std outcome: 5.345327463311591


## 3. RPS for profiles with data

We now search for the optimal theta as given by a normalized loss and chosen epsilon. Need to already specify H and the regularization parameter.

In [70]:
lambda_r = 0.3
eps = 0.05 # chosen tolerance

In [71]:
import numpy as np
from rashomon.hasse import enumerate_policies, enumerate_profiles, policy_to_profile
from rashomon.aggregate import (
    RAggregate_profile,
    find_feasible_combinations,
    subset_data,
    find_profile_lower_bound
)
from rashomon import loss

In [72]:
# Step 1: Enumerate profiles and map policies to each
profiles, profile_map = enumerate_profiles(M)
all_policies = enumerate_policies(M, R_vec)

profile_to_policies = {}
profile_to_indices = {}
for i, pol in enumerate(all_policies):
    pid = profile_map[policy_to_profile(pol)]
    profile_to_policies.setdefault(pid, []).append(pol)
    profile_to_indices.setdefault(pid, []).append(i)

We now filter for the profiles just with any data.

In [73]:
# Step 2: Filter profiles with data and compute normalized lower-bound losses
valid_pids = []
lb_k = []  # normalized lower-bound loss for each valid profile

for pid, profile in enumerate(profiles):
    Dk, yk = subset_data(D1, y1, profile_to_indices[pid])
    if Dk is None:
        continue
    mask = np.array(profile, dtype=bool)
    reduced_policies = [tuple(np.array(p)[mask]) for p in profile_to_policies[pid]]
    pm = loss.compute_policy_means(Dk, yk, len(reduced_policies))
    raw_lb = find_profile_lower_bound(Dk, yk, pm)
    lb_k.append(raw_lb / N1)
    valid_pids.append(pid)

lb_k = np.array(lb_k)                   # array of normalized lower bounds
best_loss = lb_k.min()                 # best profile loss
total_lb = lb_k.sum()
theta_global = total_lb * (1 + eps) # Theta is in reference to total loss here, not a relative value
print(f"best_loss = {best_loss:.5f}")
print(f"theta_global = {theta_global:.5f}")

best_loss = 0.09952
theta_global = 25.23455


We now construct the RPS for each profile with data from our first allocation.

In [74]:
R_profiles = []
loss_args = []

for i, pid in enumerate(valid_pids):
    profile_mask = np.array(profiles[pid], dtype=bool)
    M_k = profile_mask.sum()
    R_k = R_vec[profile_mask]

    Dk, yk = subset_data(D1, y1, profile_to_indices[pid])
    reduced_policies = [tuple(np.array(p)[profile_mask]) for p in profile_to_policies[pid]]
    pm = loss.compute_policy_means(Dk, yk, len(reduced_policies))

    theta_k = max(0.0, theta_global - (total_lb - lb_k[i]))

    print(f"Calling RAggregate_profile on profile {pid}, M_k={M_k}, len(policies)={len(reduced_policies)}, theta_k={theta_k:.5f}")
    print(f": lower_bound: {lb_k[i]:.5f}, theta_k: {theta_k:.5f}")

    rp = RAggregate_profile(
        M=M_k,
        R=R_k,
        H=H,
        D=Dk,
        y=yk,
        theta=theta_k,
        profile=tuple(profiles[pid]),
        reg=lambda_r,
        policies=reduced_policies,
        policy_means=pm,
        normalize=N1
    )

    print(f": RPS size for profile {pid}: {len(rp)}")
    if len(rp) > 0:
        R_profiles.append(rp)
        loss_args.append((Dk, yk, reduced_policies, pm))

Calling RAggregate_profile on profile 1, M_k=1, len(policies)=2, theta_k=1.85380
: lower_bound: 0.65216, theta_k: 1.85380
: RPS size for profile 1: 2
Calling RAggregate_profile on profile 2, M_k=1, len(policies)=2, theta_k=1.55939
: lower_bound: 0.35774, theta_k: 1.55939
: RPS size for profile 2: 2
Calling RAggregate_profile on profile 3, M_k=2, len(policies)=4, theta_k=2.28029
: lower_bound: 1.07864, theta_k: 2.28029
: RPS size for profile 3: 4
Calling RAggregate_profile on profile 4, M_k=1, len(policies)=2, theta_k=1.30117
: lower_bound: 0.09952, theta_k: 1.30117
: RPS size for profile 4: 2
Calling RAggregate_profile on profile 5, M_k=2, len(policies)=4, theta_k=2.28977
: lower_bound: 1.08812, theta_k: 2.28977
: RPS size for profile 5: 4
Calling RAggregate_profile on profile 6, M_k=2, len(policies)=4, theta_k=1.79353
: lower_bound: 0.59188, theta_k: 1.79353
: RPS size for profile 6: 4
Calling RAggregate_profile on profile 7, M_k=3, len(policies)=8, theta_k=3.94933
: lower_bound: 2.74

In [75]:
# Compute loss only for nonempty profile RPSs
for rp, (Dk, yk, policies_k, pm_k) in zip(R_profiles, loss_args):
    rp.calculate_loss(Dk, yk, policies_k, pm_k, lambda_r, normalize=N1)

## 4. Construct the full RPS

We demonstrate the creation of the full RPS from the profile-specific partitions.

In [59]:
from observed_RPS import RAggregate_observed_space
R_set_obs, R_profiles_obs, valid_pids, profiles = RAggregate_observed_space(
    M, R_vec, H, D1, y1, lambda_r, 0.05, True
)


print(f"\nNumber of feasible global Rashomon sets: {len(R_set_obs)}")
print(f"Number of observed profiles: {len(valid_pids)}")
for i, pid in enumerate(valid_pids):
    print(f"Profile {pid}: size {len(R_profiles_obs[i])}")

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

We demonstrate a main wrapper call, as we would use from the original Rashomon module, but we note that the function operates unexpectedly because we don't have data for a number of the profiles. (We end up with an empty Rashomon Partition Set).

In [45]:
# Doesn't work!! Because only have data on a small subspace, and quits out when we can't make a pooling decision
# Call main RAggregate function
R_set, R_profiles = RAggregate(
    M=M,
    R=R_vec,
    H=H,
    D=D1,
    y=y1,
    theta=theta,
    reg=lambda_r,
    verbose=True,
)

Skipping profile (0, 0, 0, 0)
(0, 0, 0, 1) 1.8538012790890122
Adaptive
Profile (0, 0, 0, 1) took 9.512901306152344e-05 s adaptively
Profile (0, 0, 0, 1) has 0 objects in Rashomon set
(0, 0, 1, 0) 1.5593901884315748
Adaptive
Profile (0, 0, 1, 0) took 3.814697265625e-05 s adaptively
Profile (0, 0, 1, 0) has 0 objects in Rashomon set
(0, 0, 1, 1) 2.28028641407721
Adaptive
Profile (0, 0, 1, 1) took 5.0067901611328125e-05 s adaptively
Profile (0, 0, 1, 1) has 0 objects in Rashomon set
(0, 1, 0, 0) 1.3011691608719111
Adaptive
Profile (0, 1, 0, 0) took 3.361701965332031e-05 s adaptively
Profile (0, 1, 0, 0) has 0 objects in Rashomon set
(0, 1, 0, 1) 2.289768913383508
Adaptive
Profile (0, 1, 0, 1) took 3.504753112792969e-05 s adaptively
Profile (0, 1, 0, 1) has 0 objects in Rashomon set
(0, 1, 1, 0) 1.7935284046560085
Adaptive
Profile (0, 1, 1, 0) took 3.218650817871094e-05 s adaptively
Profile (0, 1, 1, 0) has 0 objects in Rashomon set
(0, 1, 1, 1) 3.9493318341276833
Adaptive
Profile (0, 1, 1